# Web Scraping for Indeed.com & Predicting Salaries

In this project, I scrape salaries from Indeed.com, and use the features from Indeed.com to be able to predict future salaries. 

I am going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job I will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings do not come with salary information, being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression. While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries. 

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

## Problem statement

The aim of this project will be to determine what effects salary for Data Science jobs in the United States. The salary data that is found can be combined with data on location and job titles. It would be interesting to see how location and certain words in a job title (such as senior or manager) affects the probability that one gets a salary above the median for data scientists.

In [1]:
import numpy as np
import pandas as pd
import urllib2

In [2]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup

In [4]:
## YOUR CODE HERE
response = requests.get(URL)

In [5]:
response

<Response [200]>

In [6]:
page = response.text

In [7]:
soup = BeautifulSoup(page, "lxml")

In [8]:
soup

<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta content="text/html;charset=unicode-escape" http-equiv="content-type"/>\n<!-- pll --><script src="/s/7331891/en_US.js" type="text/javascript"></script>\n<link href="/s/162f93c/jobsearch_all.css" rel="stylesheet" type="text/css"/>\n<link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&amp;l=New+York" rel="alternate" title="Data Scientist $20,000 Jobs, Employment in New York, NY" type="application/rss+xml"/>\n<link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=New+York" media="handheld" rel="alternate"/>\n<script type="text/javascript">\n    \n    window['closureReadyCallbacks'] = [];\n\n    function call_when_jsall_loaded(cb) {\n        if (window['closureReady']) {\n            cb();\n        } else {\n            window['closureReadyCallbacks'].push(cb);\n        }\n    }\n</script>\n<script src="/s/a210519/jobsearch-all-compiled.js" type="text/javascript"></script>\n<script type="text/javascript">\nvar pingUrlsForGA = []

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

In [9]:
body = soup.find('body')
print type(body)

results = body.find_all('div', 'row','result' )

<class 'bs4.element.Tag'>


In [10]:
job =[]
company=[]
location=[]
salary=[]

for row in results:
    try:
        job.append(row.a.getText())
    except:
        job.append(np.nan)
    try:
        company.append(row.span.getText().strip())
    except:
        company.append(np.nan)
    try:
        location += [r.getText().strip() for r in row('span', class_='location')]
    except:
        location += [np.nan]
    try:
        salary.append(row.nobr.getText())
    except:
        salary.append(np.nan)

In [11]:
company

[u'Galvanize',
 u'Cumulus Media',
 u'United Nations',
 u'Yodle',
 u'7Park Data',
 u'Morgan Stanley',
 u'JPMorgan Chase',
 u'EXPERIAN',
 u'MassMutual Financial Group',
 u'Citi']

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

In [12]:
df_dic = {}
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Boston']):
    print city

Houston
Boston
Phoenix
Chicago
San+Francisco
New+York
Dallas
Philadelphia
Denver
Los+Angeles
Pittsburgh
Miami
Atlanta
Seattle
Austin
Portland


In [12]:
df_dic = {}
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Boston']):
    print city
    city_dic = {}
    scrape_range = range(0,1000,10)
    while scrape_range != []:
        for page in scrape_range: # page from 1 to 1010 (last page we can scrape is 1010)
            try:
                print scrape_range
                job=[]
                company=[]
                salary=[]
                location=[]
                url = "http://www.indeed.com/jobs?q=data+scientist+$20,000&l=%s&start=%s" % (city,page)
                pval = urllib2.urlopen(url).read()
                soup = BeautifulSoup(pval, 'lxml')
                resultcol = soup.findAll('div', "row")
                for row in resultcol:
                    try:
                        job.append(row.a.getText())
                    except:
                        job.append('0')
                    try:
                        company.append(row.span.getText().strip())
                    except:
                        company.append('0')
                    try:
                        location += [r.getText().strip() for r in row('span', class_='location')]
                    except:
                        location += ['0']
                    try:
                        salary.append(row.nobr.getText())
                    except:
                        salary.append(np.nan)
                if page == 0:
                    ref_job = job
                    ref_company = company
                    ref_location = location
                    df = pd.DataFrame({'Job':job,'Company':company,'Location':location,'Salary':salary})
                    ref_df = df
                    city_dic[city+'_'+str(page)] = df
                    scrape_range.remove(page)
                elif (job!=ref_job)&(company!=ref_company)&(location!=ref_location):
                    df = pd.DataFrame({'Job':job,'Company':company,'Location':location,'Salary':salary})
                    city_dic[city+'_'+str(page)] = df
                    scrape_range.remove(page)
            except:
                scrape_range.remove(page)
    data = pd.concat(city_dic.values())
    data = data.reset_index(drop=True)
    df_dic[city] = data
    print 'Finished scraping %s' % (city)

df_dic

Houston
[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990]
[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990]
[10, 20, 30, 40

{'Atlanta':                                       Company  \
 0                            Emory University   
 1                     Choice Technology Group   
 2                               Atkins Global   
 3           Oak Ridge Associated Universities   
 4                                         ICF   
 5                                   Honeywell   
 6                                   Honeywell   
 7             Georgia Tech Research Institute   
 8                              SimplePart.com   
 9                                    Deloitte   
 10                         Intuitive Surgical   
 11                                   CocaCola   
 12                               Immucor, Inc   
 13                             Black & Veatch   
 14       Groundwater & Environmental Services   
 15                          Quest Diagnostics   
 16                                 Alcon Labs   
 17                            G2S Corporation   
 18            Georgia Institute of Tec

In [120]:
df_dic['Chicago'].head()

,Company,Job,Location,Salary
0,Trunk Club,Director of Data Science,"Chicago, IL 60654 (Loop area)",NaN
1,Ipsos North America,USPA - Statistical Programmer,"Chicago, IL",NaN
2,University of Chicago,Research Data Analyst,"Chicago, IL",NaN
3,Ipsos North America,Connect - Senior Research Analyst,"Chicago, IL",NaN
4,Amyx,SEC Financial Computer Scientist (Python),"Chicago, IL",NaN


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [13]:
final_df = df_dic['Chicago']

In [14]:
cities = (['New+York', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY])

In [15]:
#for i in cities:
for i in cities:
    final_df= final_df.append(df_dic[i])

In [16]:
final_df.describe()

,Company,Job,Location,Salary
count,12334,12334,12334,670
unique,2354,5195,729,306
top,"Magic Leap, Inc.",Data Scientist,"Houston, TX","$50,000 a year"
freq,577,369,698,94


In [17]:
df_sal = final_df.dropna()

In [18]:
df_sal.head()

,Company,Job,Location,Salary
29,Modis,Statistical Data Analyst- Long-term Contract i...,"Northbrook, IL 60062",$25 - $35 an hour
30,S.C. International,Statistical Analyst -- Health-7444 - 7444,"Chicago, IL","$80,000 a year"
88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)","Chicago, IL","$110,000 - $155,000 a year"
95,Workbridge Associates,Data Scientist (Healthcare Data),"Chicago, IL","$110,000 a year"
103,Jobspring Partners,Quality Engineer (Big Data),"Chicago, IL","$70,000 - $105,000 a year"


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

In [19]:
## YOUR CODE HERE
df_sal = df_sal[df_sal.Salary.str.contains("month") == False]
df_sal = df_sal[df_sal.Salary.str.contains("hour") == False]
df_sal = df_sal[df_sal.Salary.str.contains("day") == False]
df_sal = df_sal[df_sal.Salary.str.contains('week') == False]
df_sal = df_sal.drop_duplicates()

In [20]:
df_sal['Location']= df_sal['Location'].astype(str)

In [21]:
for i in df_sal.index:
    if 'Chicago'in df_sal.loc[i, 'Location']:
        df_sal.loc[i, 'Location']= 'Chicago'

In [22]:
df_sal.head()

,Company,Job,Location,Salary
30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,"$80,000 a year"
88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,"$110,000 - $155,000 a year"
95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,"$110,000 a year"
103,Jobspring Partners,Quality Engineer (Big Data),"Chicago, IL","$70,000 - $105,000 a year"
114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,"$100,000 - $130,000 a year"


In [23]:
df_sal['Salary'] = df_sal['Salary'].map(lambda x: x.lstrip('$').rstrip('a year'))

### Save your results as a CSV

In [24]:
df_sal[['Salary', 'Salary_2']]= df_sal['Salary'].str.split('$', expand=True)
df_sal = df_sal.fillna(value='0')

In [25]:
df_sal['Salary'] = df_sal['Salary'].map(lambda x: x.rstrip(' -'))

In [27]:
df_sal.head()

,Company,Job,Location,Salary,Salary_2
30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000,0
88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,110000,"155,000"
95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000,0
103,Jobspring Partners,Quality Engineer (Big Data),"Chicago, IL",70000,"105,000"
114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,100000,"130,000"


In [28]:
df_sal['Salary_2'] = df_sal['Salary_2'].map(lambda x: int(x.replace(',', '')))

In [29]:
df_sal['Sal_avg']=0
for i in range(len(df_sal['Salary'])):
    if df_sal.iloc[i, 4]==0:
        df_sal.iloc[i, 5]= df_sal.iloc[i,3]
    else:
        df_sal.iloc[i, 5]= (df_sal.iloc[i,3]+df_sal.iloc[i,4])/2

In [190]:
df_sal.to_csv('Salaries.csv', sep=';', encoding='utf-8')

In [33]:
df_sal

,Company,Job,Location,Salary,Salary_2,Sal_avg
30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000,0,80000
88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,110000,155000,132500
95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000,0,110000
103,Jobspring Partners,Quality Engineer (Big Data),"Chicago, IL",70000,105000,87500
114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,100000,130000,115000
128,Hirewell,Data Scientist -Fitness/Wellness Firm,Chicago,120000,0,120000
133,Smith Hanley Associates,Statistical Research Scientist,Chicago,75000,90000,82500
158,Workbridge Associates,Data Scientist,Chicago,125000,0,125000
163,U.S. Department of Labor,Economist/Statistician (Economics),Chicago,43915,69827,56871
334,GinasTechJobs.com,Associate Quantitative Research Analyst,Chicago,180000,0,180000


As a result of the scraping, there is now a clean dataset with a column that takes the average of a range if a range is given, and which takes the value given otherwise. This dataset is saved to a csv for later use.

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [59]:
import pandas as pd
import numpy as np

In [60]:
df = pd.read_csv('Salaries.csv', sep=';')

In [61]:
df = df.drop(['Salary'], axis=1)
df = df.drop(['Salary_2'], axis=1)

In [62]:
df.head()

,Unnamed: 0,Company,Job,Location,Sal_avg
0,30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000
1,88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,132500
2,95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000
3,103,Jobspring Partners,Quality Engineer (Big Data),"Chicago, IL",87500
4,114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,115000


In [63]:
df['Location'].value_counts()

New York                                          62
Chicago                                           20
San Francisco, CA                                 19
Boston, MA                                        18
Atlanta, GA                                       17
Los Angeles, CA                                   17
New York, NY                                      13
Coral Gables, FL                                  11
Manhattan, NY                                     10
Cambridge, MA                                     10
Philadelphia, PA                                   9
Seattle, WA                                        7
Chicago, IL                                        7
Phoenix, AZ                                        7
Austin, TX                                         7
Houston, TX                                        7
Denver, CO                                         5
Pittsburgh, PA                                     5
Boston, MA 02116 (South End area)             

Since the locations in these search results are not very clean, I use a function to find inputs in the dataframe that exactly match (part of) a location name.

In [64]:
city_no_spaces = (['York', 'Francisco', 'Austin', 'Seattle', 
    'Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Chicago', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Boston'])

In [65]:
for city in city_no_spaces:
    for i in df.index:
        if city in df.loc[i, 'Location']:
            df.loc[i, 'Location']= city

In [66]:
df = df.loc[df['Location'].isin(city_no_spaces)]


In [67]:
df.head()

,Unnamed: 0,Company,Job,Location,Sal_avg
0,30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000
1,88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,132500
2,95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000
3,103,Jobspring Partners,Quality Engineer (Big Data),Chicago,87500
4,114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,115000


I want to predict a binary variable, whether the salary is high or low. We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a binary classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the median as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries.

In [68]:
med_salary= np.median(df['Sal_avg'])
print med_salary

105000.0


In [69]:
for i in df.index:
    if df.loc[i, 'Sal_avg']>med_salary:
        df.loc[i,'high_salary']=int(1) 
    else: 
        df.loc[i,'high_salary']=int(0)

In [70]:
df.head()

,Unnamed: 0,Company,Job,Location,Sal_avg,high_salary
0,30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000,0.0
1,88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,132500,1.0
2,95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000,1.0
3,103,Jobspring Partners,Quality Engineer (Big Data),Chicago,87500,0.0
4,114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,115000,1.0


I run a model to get a baseline accuracy for the classification problem.

In [71]:
import statsmodels.formula.api as smf

model1 = smf.logit(formula="high_salary ~ 1",data = df).fit() 
model1.summary() 

Optimization terminated successfully.
         Current function value: 0.692964
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            high_salary   No. Observations:                  261
Model:                          Logit   Df Residuals:                      260
Method:                           MLE   Df Model:                            0
Date:                Wed, 18 Jan 2017   Pseudo R-squ.:                   0.000
Time:                        17:26:57   Log-Likelihood:                -180.86
converged:                       True   LL-Null:                       -180.86
                                        LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.0383      0.124     -0.309      0.757        -0.281     0.204
==============================================================================
"""

In [72]:
outcomes = df["high_salary"].value_counts()

In [73]:
outcomes

0.0    133
1.0    128
Name: high_salary, dtype: int64

In [74]:
from __future__ import division

In [75]:
prob_high_salary=outcomes[1]/(outcomes[0]+outcomes[1])
print(prob_high_salary)

odds_high_salary=prob_high_salary/(1-prob_high_salary)
print(odds_high_salary)

log_odds_high_salary=np.log(odds_high_salary)
print(log_odds_high_salary)

0.490421455939
0.962406015038
-0.0383188643021


Since the criteria for getting a high salary is to be above the median, there is a slightly bigger chance to have a low salary (since salaries with exactly the median value get put into the low salary category).

In [76]:
model1 = smf.logit(formula="high_salary ~ Location", data = df).fit() 
model1.summary() 

         Current function value: 0.550270
         Iterations: 35


/Users/vincentbolwerk/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            high_salary   No. Observations:                  261
Model:                          Logit   Df Residuals:                      245
Method:                           MLE   Df Model:                           15
Date:                Wed, 18 Jan 2017   Pseudo R-squ.:                  0.2059
Time:                        17:27:01   Log-Likelihood:                -143.62
converged:                      False   LL-Null:                       -180.86
                                        LLR p-value:                 7.010e-10
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                   -1.2528      0.567     -2.210      0.027        -2.364    -0.142
Location[T.Atlanta]          0.1542      0.767      0.201      0.841        -1.349     1.657
Location[T.Austin]           0.7419      0.925      0.802      0.422        -1.070     2.554
Location[T.Boston]           1.4069      0.690      2.039      0.041         0.054     2.759
Location[T.Chicago]          1.4759      0.687      2.150      0.032         0.130     2.822
Location[T.Dallas]          -0.1335      1.254     -0.107      0.915        -2.590     2.323
Location[T.Denver]          -0.6931      1.210     -0.573      0.567        -3.065     1.679
Location[T.Francisco]       24.2105   2.16e+04      0.001      0.999     -4.23e+04  4.24e+04
Location[T.Houston]          0.1542      0.994      0.155      0.877        -1.794     2.102
Location[T.Miami]          -20.8886   3.71e+04     -0.001      1.000     -7.27e+04  7.27e+04
Location[T.Philadelphia]     1.4351      0.830      1.730      0.084        -0.191     3.061
Location[T.Phoenix]          0.1542      0.875      0.176      0.860        -1.561     1.869
Location[T.Pittsburgh]     -52.7460   2.38e+11  -2.22e-10      1.000     -4.66e+11  4.66e+11
Location[T.Portland]       -20.8886   3.71e+04     -0.001      1.000     -7.27e+04  7.27e+04
Location[T.Seattle]          3.0445      1.220      2.496      0.013         0.654     5.435
Location[T.York]             1.6582      0.611      2.713      0.007         0.460     2.856
============================================================================================
"""

As can be seen from the logit regression above, the probability of getting a salary above or below the median is dependent on the location of the job title. When looking at the statistically significant p-values at the 5%-level, it is interesting to see that all of them have a positive statistically significant effect (the most significant seem to be New York and Seattle). The fact that most of the locations have a positive effect on the probability of a high salary can be explained by the fact that the intercept is negative. 

A next step is to find if certain words in a job title have a predictive effect on salary.

In [77]:
df['Job'] = df['Job'].map(lambda x: x.decode('utf-8'))

In [78]:
for i in df.index:
    if 'Senior' in df.loc[i,'Job']:
        df.loc[i,'Senior']=1
    else:
        df.loc[i, 'Senior']=0

In [79]:
for i in df.index:
    if 'Manager' in df.loc[i,'Job']:
        df.loc[i,'Manager']=1
    else:
        df.loc[i, 'Manager']=0

In [80]:
df.head()

,Unnamed: 0,Company,Job,Location,Sal_avg,high_salary,Senior,Manager
0,30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000,0.0,0.0,0.0
1,88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,132500,1.0,1.0,0.0
2,95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000,1.0,0.0,0.0
3,103,Jobspring Partners,Quality Engineer (Big Data),Chicago,87500,0.0,0.0,0.0
4,114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,115000,1.0,0.0,1.0


In [81]:
model2 = smf.logit(formula="high_salary ~ Location+ Manager + Senior", data = df).fit() 
model2.summary() 

         Current function value: 0.543869
         Iterations: 35


/Users/vincentbolwerk/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            high_salary   No. Observations:                  261
Model:                          Logit   Df Residuals:                      243
Method:                           MLE   Df Model:                           17
Date:                Wed, 18 Jan 2017   Pseudo R-squ.:                  0.2152
Time:                        17:27:06   Log-Likelihood:                -141.95
converged:                      False   LL-Null:                       -180.86
                                        LLR p-value:                 9.300e-10
============================================================================================
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                   -1.3559      0.575     -2.359      0.018        -2.482    -0.229
Location[T.Atlanta]          0.2116      0.772      0.274      0.784        -1.301     1.725
Location[T.Austin]           0.6379      0.937      0.681      0.496        -1.198     2.474
Location[T.Boston]           1.3249      0.697      1.900      0.057        -0.042     2.692
Location[T.Chicago]          1.5141      0.693      2.186      0.029         0.157     2.871
Location[T.Dallas]          -0.2165      1.267     -0.171      0.864        -2.699     2.266
Location[T.Denver]          -0.6733      1.236     -0.545      0.586        -3.095     1.749
Location[T.Francisco]       27.5351   1.18e+05      0.000      1.000     -2.32e+05  2.32e+05
Location[T.Houston]          0.0338      1.008      0.034      0.973        -1.941     2.009
Location[T.Miami]          -20.7823    3.7e+04     -0.001      1.000     -7.26e+04  7.26e+04
Location[T.Philadelphia]     1.4034      0.837      1.677      0.094        -0.237     3.044
Location[T.Phoenix]          0.1285      0.892      0.144      0.886        -1.621     1.878
Location[T.Pittsburgh]     -18.7190   9190.774     -0.002      0.998      -1.8e+04   1.8e+04
Location[T.Portland]       -20.7823    3.7e+04     -0.001      1.000     -7.26e+04  7.26e+04
Location[T.Seattle]          3.1477      1.224      2.573      0.010         0.750     5.546
Location[T.York]             1.6611      0.616      2.695      0.007         0.453     2.869
Manager                      0.3065      0.783      0.391      0.695        -1.228     1.841
Senior                       0.7840      0.442      1.773      0.076        -0.083     1.651
============================================================================================
"""

As can be seen from the regression results above, only the word 'Senior' in the job title seems to have a statistically significant positive effect at the 10%-level, whereas the word 'Manager' in the job title does not. Using the word 'Senior' thus does provide some additional explanatory power to our model.

In [82]:
df = pd.concat([df, pd.get_dummies(df['Location'])], axis=1)

In [83]:
df.head()

,Unnamed: 0,Company,Job,Location,Sal_avg,high_salary,Senior,Manager,Angeles,Atlanta,...,Denver,Francisco,Houston,Miami,Philadelphia,Phoenix,Pittsburgh,Portland,Seattle,York
0,30,S.C. International,Statistical Analyst -- Health-7444 - 7444,Chicago,80000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,88,Workbridge Associates,"Senior Data Scientist (H20, Python, and R)",Chicago,132500,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,95,Workbridge Associates,Data Scientist (Healthcare Data),Chicago,110000,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,103,Jobspring Partners,Quality Engineer (Big Data),Chicago,87500,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,114,Analytic Recruiting,Manager / Director of Predictive Modeling,Chicago,115000,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, I try to set up the same model using sci-kit learn, using a simple logistic regression.

In [94]:
X = pd.DataFrame(df.loc[:, 'Senior':])
y = df['high_salary']
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

model = LogisticRegression() 

model.fit(X,y)
print model.score(X,y)

print cross_val_score(model, X,y).mean()

0.689655172414
0.386973180077


The model seems to overfit quite dramatically, which leads to poor performance when having a cross-validated score. Perhaps doing a Gridsearch while adding a penalty function will improve the performance.

In [90]:
from sklearn.model_selection import GridSearchCV
params = {'C': [0.01, 0.1, 1, 10, 100], 'penalty':['l1', 'l2']}
logreg_cv = GridSearchCV(model, param_grid=params, cv=5)

In [91]:
logreg_cv.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [92]:
logreg_cv.best_score_

0.61685823754789271

In [93]:
logreg_cv.best_params_

{'C': 0.01, 'penalty': 'l2'}

In [46]:
def examine_coefficients(model, df):
    df = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : df.columns}
    ).sort_values(by='Coefficient')
    return df[df.Coefficient !=0 ]

In [47]:
examine_coefficients(model, X)

,Coefficient,Feature
14,-1.030806,Pittsburgh
8,-0.811404,Denver
15,-0.709577,Portland
11,-0.709577,Miami
2,-0.666508,Angeles
3,-0.533948,Atlanta
7,-0.511610,Dallas
13,-0.508915,Phoenix
10,-0.497960,Houston
4,-0.139108,Austin


In [48]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, y))

             precision    recall  f1-score   support

        0.0       0.63      0.72      0.67       116
        1.0       0.75      0.66      0.70       145

avg / total       0.70      0.69      0.69       261



### Conclusion

The values given by this model indicate that our model is an improvement of the baseline accuracy. The model achieves an accuracy of 61.6%, which is an improvement on the earlier accuracy of approximately 50%. However, it was very important to do cross-validation, and to add a penalty function to our logistic regression.

The most important features in determining a data scientist's salary are the location of the job (where obviously San Francisco has the largest positive effect on salary). Next to that, being Senior is also an important factor in determining your future salary.